In [1]:
import ghmm
from collections import OrderedDict
import cPickle as pickle
import numpy as np
from itertools import product as iterproduct, chain
from pprint import pprint
import pysam
import os
import pandas
from copy import deepcopy
import re
import editdistance
import sys
import math
import random
from nbwrapper import getargs
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/alignment_validation.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/alignment_lib.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/analysis_lib.ipynb"
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/lib/feature_lib.ipynb"

In [2]:
args = getargs()

/home/ibis/gregor.sturm/bin/anaconda/lib/python2.7/site-packages/nbwrapper.py:31: RuntimeWarning: no arguments passed!
  warnings.warn("no arguments passed!", RuntimeWarning)


In [3]:
NMERS = 6

In [4]:
# args = {
#     "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1_100.events.template.pickle",
#     "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1_100.called",
#     "ref": "/home/ibis/gregor.sturm/nanopore/NanoporeData/PublicData/LomanLab_MAP-006/ecoli_mg1655.fa",
#     "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1.model.pickle",
#     "corr_model": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/context_prediction/models/model-test2.pickle",
#     "ncores": 62,
#     "nmers": NMERS,
#     "multivariate": False
# }
args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/wouter_lambda006_100.events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1_100.called",
    "ref": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_ref.fasta",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/loman006-1.model.pickle",
    "corr_model": "/home/ibis/gregor.sturm/nanopore/own/notebooks/05_MAP006-basecaller/context_prediction/models/model-test2.pickle",
    "ncores": 62,
    "nmers": NMERS,
    "multivariate": False
}


In [5]:
NMERS = int(args["nmers"])
NSTATES = 4**NMERS
MULTIVARIATE = bool(int(args["multivariate"]))
args["ncores"] = int(args["ncores"])

In [6]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = HMM_PARAMS["/opt/chimaera/model/r7.3_e6_70bps_6mer/template_median68pA.model"]
ALL_KMERS = ["".join(x) for x in iterproduct("ACGT", repeat=NMERS)]
assert HMM_PARAMS["kmer"].tolist() == ALL_KMERS

# Train Model 

In [7]:
def mk_transmat1(nmers):
    """make a transition matrix assuming move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [8]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [9]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    n_components = len(ALL_KMERS)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(ALL_KMERS):
        for i, to_kmer in enumerate(ALL_KMERS):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (2/50.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (47/50.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/50.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [10]:
mk_transmat = mk_transmat2

In [11]:
def mk_model_simple(): 
    """ simple model, only taking the means into account. """
    A = mk_transmat(NMERS)
    B = HMM_PARAMS[["level_mean", "level_stdv"]].values.tolist() #mu, std of each state
    pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state
    # generate model from parameters
    model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)
    return model

In [12]:
F = ghmm.Float()  # emission domain of this model
def mk_model():
    if MULTIVARIATE: 
        return mk_model_multivariate()
    else: 
        return mk_model_simple()

In [13]:
model = mk_model()
s = str(model)
print(s)

GaussianEmissionHMM(N=4096)
  state 0 (initial=0.00, mu=62.78, sigma=0.84)
    Transitions: ->0 (0.00), ->1 (0.00), ->2 (0.00), ->3 (0.00), ->4 (0.00), ->5 (0.00), ->6 (0.00), ->7 (0.00), ->8 (0.00), ->9 (0.00), ->10 (0.00), ->11 (0.00), ->12 (0.00), ->13 (0.00), ->14 (0.00), ->15 (0.00)
  state 1 (initial=0.00, mu=58.02, sigma=0.66)
    Transitions: ->1 (0.02), ->4 (0.23), ->5 (0.23), ->6 (0.23), ->7 (0.23), ->16 (0.00), ->17 (0.00), ->18 (0.00), ->19 (0.00), ->20 (0.00), ->21 (0.00), ->22 (0.00), ->23 (0.00), ->24 (0.00), ->25 (0.00), ->26 (0.00), ->27 (0.00), ->28 (0.00), ->29 (0.00), ->30 (0.00), ->31 (0.00)

  ...

  state 4094 (initial=0.00, mu=45.36, sigma=0.64)
    Transitions: ->4064 (0.00), ->4065 (0.00), ->4066 (0.00), ->4067 (0.00), ->4068 (0.00), ->4069 (0.00), ->4070 (0.00), ->4071 (0.00), ->4072 (0.00), ->4073 (0.00), ->4074 (0.00), ->4075 (0.00), ->4076 (0.00), ->4077 (0.00), ->4078 (0.00), ->4079 (0.00), ->4088 (0.23), ->4089 (0.23), ->4090 (0.23), ->4091 (0.23), ->409

In [14]:
def result_to_seq(result):
    states = result[0]
    kmers = [ALL_KMERS[x] for x in states]
    seq = [kmer[0] for kmer in kmers] + [kmers[-1][1:]]
    return "".join(seq)

In [15]:
def predict(events):
    """mixed is a set of tuples (event_mean, event_stdv)"""
    emissions = [x[0] for x in events]
    seq = ghmm.EmissionSequence(F, emissions)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [16]:
s = model.sampleSingle(10)
s = [x for x in s]
seq = zip([s[i] for i in range(0, len(s), 2)], [s[i] for i in range(1, len(s), 2)])

In [17]:
predict(seq)

'CGGAGCGTGC'

In [18]:
model

## multistep-prediction

In [19]:
from sklearn import ensemble
import joblib
import mltools

In [20]:
corr_model = joblib.load(args["corr_model"])

In [21]:
OFFSET = 20

In [22]:
def get_correction(features):
    correction = corr_model.predict(features)
    i_min = np.argmin([abs(x) for x in correction])
    correction = mltools.normalize_between(correction, -.5, .5)
    shift = correction[i_min]
    correction = [x - shift for x in correction]
    return correction

In [23]:
def get_features(events, seq): 
    corr_range = (OFFSET, len(seq)-OFFSET-NMERS)
 
    features = []
    for i in range(*corr_range):
        mean, stdv = events[i]
        context = seq[i-OFFSET:i+NMERS+OFFSET]
        assert seq[i:i+6] == context[20:26]
        features.append(mk_feature_all(mean, stdv, context))

    features = postprocess_features(features)
    
    return features

In [24]:
def correct_events(events, seq): 
    corr_range = (OFFSET, len(seq)-OFFSET-NMERS)
 
    features = get_features(events, seq)
    correction = get_correction(features)
    for i, j  in enumerate(range(*corr_range)):
        mean, stdv = events[j]
        tmp_mean = mean - correction[i]
        ratio = tmp_mean/mean
        tmp_stdv = ratio * stdv
        events[j] = tmp_mean, tmp_stdv

    return events

In [25]:
def predict_iterative(events, n_steps=5): 
    for _ in range(n_steps):
        seq = predict(events)
        if _ < n_steps-1: 
            events = correct_events(events, seq)
    return seq

In [26]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [27]:
file_obj = correct_read(file_data[0], col="mean")
events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]

In [28]:
seq = predict(events)

In [29]:
seq[18:40]

'GTGTGTTGCGGCGTCTCGGTCA'

In [30]:
events[18:30]

[(55.31206783664395, 1.3709859354515133),
 (51.90241473405205, 1.438436609021339),
 (46.98130819199456, 1.6793134137960855),
 (54.460153893576134, 0.6459907934079204),
 (61.67123592293892, 1.4355407958164157),
 (53.47931387800489, 2.1127559715227684),
 (56.75266861105184, 2.170003935918162),
 (44.67298491803681, 1.4567948577847787),
 (54.41486104961385, 1.562182944220163),
 (62.22543778766402, 1.3647850791960336),
 (46.42318261701293, 1.6552373305698),
 (51.08394350484226, 1.4034213888168883)]

In [31]:
features = get_features(events, seq)
[x for x in enumerate(features[0])]

[(0, 0.16926698960201006),
 (1, 0.3402043334463796),
 (2, 0),
 (3, 0),
 (4, 1),
 (5, 0),
 (6, 1),
 (7, 0),
 (8, 0),
 (9, 0),
 (10, 0),
 (11, 1),
 (12, 0),
 (13, 0),
 (14, 0),
 (15, 1),
 (16, 0),
 (17, 0),
 (18, 0),
 (19, 0),
 (20, 1),
 (21, 0),
 (22, 0),
 (23, 0),
 (24, 0),
 (25, 1),
 (26, 0),
 (27, 0),
 (28, 0),
 (29, 1),
 (30, 1),
 (31, 0),
 (32, 0),
 (33, 0),
 (34, 0),
 (35, 0),
 (36, 0),
 (37, 1),
 (38, 0),
 (39, 1),
 (40, 0),
 (41, 0),
 (42, 0),
 (43, 0),
 (44, 0),
 (45, 1),
 (46, 0),
 (47, 0),
 (48, 1),
 (49, 0),
 (50, 0),
 (51, 0),
 (52, 1),
 (53, 0),
 (54, 1),
 (55, 0),
 (56, 0),
 (57, 0),
 (58, 0),
 (59, 0),
 (60, 0),
 (61, 1),
 (62, 0),
 (63, 0),
 (64, 0),
 (65, 1),
 (66, 0),
 (67, 1),
 (68, 0),
 (69, 0),
 (70, 0),
 (71, 0),
 (72, 0),
 (73, 1),
 (74, 0),
 (75, 0),
 (76, 1),
 (77, 0),
 (78, 0),
 (79, 0),
 (80, 0),
 (81, 1),
 (82, 0),
 (83, 0),
 (84, 1),
 (85, 0),
 (86, 0),
 (87, 0),
 (88, 0),
 (89, 1),
 (90, 0),
 (91, 0),
 (92, 1),
 (93, 0),
 (94, 0),
 (95, 0),
 (96, 0),
 (97,

In [32]:
correction = corr_model.predict(features)
i_min = np.argmin([abs(x) for x in correction])
correction = mltools.normalize_between(correction, -.5, .5)
shift = correction[i_min]
correction = [x - shift for x in correction]
correction

[-0.011406653916490817,
 0.099387985528890499,
 0.25390139862086347,
 -0.1424413847502855,
 -0.049952901966349827,
 -0.20129594392278444,
 -0.042251387982249589,
 0.36344406790025907,
 0.0052561032132469876,
 -0.006453296371870143,
 0.12474607722382547,
 -0.010976214050314737,
 0.11140128595150445,
 -0.072378103084942302,
 0.039598275186127241,
 0.006893615016757948,
 0.071051768791722925,
 0.097225556446980277,
 0.12575894561295803,
 0.014805868537105527,
 0.081178495334062906,
 -0.12740732669370713,
 0.11340059461104401,
 -0.036442651671835025,
 0.038437812052209308,
 0.21608323316793987,
 0.065091344237065085,
 -0.095947633312328418,
 -0.13023251030376715,
 0.062162866736274636,
 0.11056849393353302,
 0.039216534173022233,
 0.12103472218959466,
 0.1007264113074286,
 0.1493314291346115,
 -0.049694924145555097,
 0.037359952488802639,
 0.066243727787022033,
 0.013293782457567571,
 0.034852667108531632,
 0.025818938289818516,
 0.10677684687813715,
 0.012598293905504376,
 0.0537457250117

In [33]:
events = correct_events(events, seq)
events[18:30]

[(55.31206783664395, 1.3709859354515133),
 (51.90241473405205, 1.438436609021339),
 (46.992714845911046, 1.6797211365195612),
 (54.360765908047242, 0.64481188150560576),
 (61.417334524318058, 1.4296306529373182),
 (53.621755262755173, 2.118383266721612),
 (56.802621513018188, 2.1719139429104395),
 (44.874280861959598, 1.4633591582571488),
 (54.457112437596095, 1.5633959289894397),
 (61.86199371976376, 1.3568136922740934),
 (46.417926513799678, 1.6550499220863344),
 (51.090396801214133, 1.4035986792438853)]

# Validate Model 

In [281]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks


In [282]:
assert os.path.isfile(args["events"])

In [283]:
ref = load_ref(args["ref"])

['>gi|556503834|ref|NC_000913.3| Escherichia coli str. K-12 substr. MG1655, complete genome']
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAAT


In [284]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [285]:
prepare_filemap(file_data)

In [286]:
def basecall_read(file_obj):
    file_obj = correct_read(file_obj, col="mean")
    events = [(x["mean"], x["stdv"]) for x in file_obj["events"].to_dict("records")]
    called_seq = predict_iterative(events)
    return (file_obj["channel"], file_obj["file_id"], called_seq)

In [287]:
# """ train with baum-welch """
# for i, file_obj in enumerate(file_data): 
#     sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
#     train_read(file_obj)

In [288]:
p = Pool(args["ncores"])

In [ ]:
#basecall_read(file_data[3])

In [ ]:
print("Prediction: ")
results = []
try:
    for i, res in enumerate(p.imap_unordered(basecall_read, file_data), 1):
        results.append(res)
        sys.stdout.write('\rdone {0:%}'.format(i/float(len(file_data))))
    p.close()
    p.join()
except KeyboardInterrupt:
    p.terminate()

### Stats

In [ ]:
types = ["metrichor", "called", "random"]
fasta_files = {t: "{0}.{1}.fa".format(args["out_basename"], t) for t in types}

In [ ]:
## metrichor fasta
with open(fasta_files["metrichor"], 'w') as f: 
    for file_obj in file_data: 
        f.write(">ch{0}_file{1}_metrichor".format(file_obj["channel"], file_obj["file_id"])+ "\n")
        f.write(file_obj["fastq"].split("\n")[1] + "\n")

In [ ]:
## called fasta/random fasta
with open(fasta_files["called"], 'w') as f: 
    with open(fasta_files["random"], 'w') as fr:
        for channel, file_id, seq in results: 
            f.write(">ch{0}_file{1}_called".format(channel, file_id)+ "\n")
            fr.write(">ch{0}_file{1}_random".format(channel, file_id)+ "\n")
            f.write(seq + "\n")
            fr.write("".join([random.choice("ACGT") for _ in range(len(seq))]))

In [ ]:
for t in types: 
    sam_file = "{0}.{1}.sam".format(args["out_basename"], t)
    graphmap(args["ref"], fasta_files[t], sam_file, args["ncores"])
    prepare_sam("{0}.{1}".format(args["out_basename"], t))

In [ ]:
def mk_stat(t):
    samfile = "{0}.{1}.sorted.bam".format(args["out_basename"], t)
    sst = samstats(samfile, ref, ncores=args["ncores"])
    return pandas.DataFrame(sst.print_summary())

In [ ]:
stats = map(mk_stat, types)
print(types)
side_by_side(*stats)

In [ ]:
# for t, df in zip(types, stats):
#     with open("{0}.stats.{1}.html".format(args["out_basename"], t), 'w') as f:
#         f.write(df.to_html())

In [ ]:
# def score_consensus(t):
#     consensus = mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], t), ref_file)
#     return(consensus)
#     consensus = consensus.split("\n")[1].to_upper()
#     score = needle(ref, consensus)
#     return (consensus, score)

In [ ]:
# p = Pool(args["ncores"])
# try:
#     consensus = p.map(score_consensus, types)
#     p.close()
# except KeyboardInterrupt:
#     p.terminate()

In [ ]:
# consensus

In [ ]:
# mk_consensus("{0}.{1}.sorted.bam".format(args["out_basename"], "metrichor"), ref_file)